### Cookie problem

using CammelCase because of author

In [79]:
from code_.thinkbayes import Pmf, Suite

In [17]:
pmf = Pmf()

# priors
pmf.Set('B1', 0.5)
pmf.Set('B2', 0.5)
pmf.GetDict()

{'B1': 0.5, 'B2': 0.5}

In [18]:
# update based on new data (prior * likelihood)
pmf.Mult('B1', 3/4)
pmf.Mult('B2', 1/2)
# After this update, the distribution is no longer normalized, but because these hypotheses
# are mutually exclusive and collectively exhaustive, we can renormalize:
pmf.Normalize()
print(pmf.GetDict())
print(pmf.Prob('B1'))

{'B1': 0.6000000000000001, 'B2': 0.4}
0.6000000000000001


In [26]:
class Cookie(Pmf):
    def __init__(self, hypotheses, mixes=None):
        # calculate prior
        Pmf.__init__(self)
        for H in hypotheses:
            self.Set(H, 1)
        self.Normalize()

        if mixes is None:
            self.mixes = {
                'B1': dict(vanilla=0.75, chocolate=0.25),
                'B2': dict(vanilla=0.5, chocolate=0.5),
            }
        else:
            self.mixes = mixes
    
    def Likelihood(self, data, hypothesis):
        mix = self.mixes[hypothesis]
        likelihood = mix[data]
        return likelihood
    
    def Update(self, data):
        for H in self.Values():
            likelihood = self.Likelihood(data, H)
            self.Mult(H, likelihood)
        self.Normalize()
        return
    

In [27]:
cookies = Cookie(
    ['B1', 'B2']
)
cookies.GetDict()

{'B1': 0.5, 'B2': 0.5}

In [28]:
# we picked vanilla cookie from a bowl
cookies.Update('vanilla')
cookies.GetDict()

{'B1': 0.6000000000000001, 'B2': 0.4}

In [29]:
# we picked chocolate cookie from a bowl
cookies.Update('chocolate')
cookies.GetDict()

{'B1': 0.4285714285714286, 'B2': 0.5714285714285714}

# Monty Hall Problem

In [72]:
# empirically simulate, we pick door A
import numpy as np

doors = {'A', 'B', 'C'}
pick = 'A'

monty_doors = list(doors - {'A'})
steps = 5000
print(monty_doors)

['C', 'B']


In [74]:
def pick_monty_action(pick:str, car_door:str, doors:list):
    if pick == car_door:
        res = np.random.choice(doors)
    elif pick != car_door:
        posibilities = list(
            set(doors) - set(car_door)
        )
        res = np.random.choice(posibilities)
    return res

In [78]:
pick_monty_action('A', 'A', ['B', 'C'])

'B'

In [76]:
results = {
    # key is monty action; stick/switch is player action with win count
    'B': {'stick': 0, 'switch': 0},
    'C': {'stick': 0, 'switch': 0}
}


for i in range(steps):
    car_is_behind = np.random.choice(list(doors))
    monty_action = pick_monty_action(
        pick, car_is_behind, monty_doors
    )

    if car_is_behind == pick:
        results[monty_action]['stick'] += 1
    else:
        results[monty_action]['switch'] += 1

# normalize results to 0, 1 interval
for MA in results:
    norm = results[MA]['stick'] + results[MA]['switch']
    for k, v in results[MA].items():
        results[MA][k] /= norm

In [77]:
results

{'B': {'stick': 0.31983967935871743, 'switch': 0.6801603206412825},
 'C': {'stick': 0.3341317365269461, 'switch': 0.665868263473054}}

# M&M

In [82]:
mix94 = dict(
    brown=30,
    yellow=20,
    red=20,
    green=10,
    orange=10,
    tan=10
)
mix96 = dict(
    blue=24,
    green=20,
    orange=16,
    yellow=14,
    red=13,
    brown=13
)
hypoA = dict(bag1=mix94, bag2=mix96)
hypoB = dict(bag1=mix96, bag2=mix94)
hypotheses = dict(A=hypoA, B=hypoB)

In [85]:
class M_and_M(Suite):
    def __init__(self, hypotheses):
        Suite.__init__(self)
        self.hypotheses = hypotheses
        for H in hypotheses:
            self.Set(H, 1)
        self.Normalize()
        print(self.GetDict())

    def Likelihood(self, data, hypo):
        bag, color = data
        mix = self.hypotheses[hypo][bag]
        like = mix[color]
        return like

In [86]:
mam = M_and_M(hypotheses)

{'A': 0.5, 'B': 0.5}


In [87]:
mam.Update(('bag1', 'yellow'))
mam.Update(('bag2', 'green'))
mam.Print()

A 0.7407407407407407
B 0.2592592592592592
